<a href="https://colab.research.google.com/github/iskra3138/GluonTS/blob/master/TFT_favorita_hm_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Jun 10 13:03:40 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:

#from google.colab import drive

In [0]:
#drive.mount('/Gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /Gdrive


In [0]:
#cd '/Gdrive/My Drive/TFT'

/Gdrive/My Drive/TFT


In [1]:
# install to download only tft directory
!pip install git+git://github.com/HR/github-clone#egg=ghclone

  Cloning git://github.com/HR/github-clone to /tmp/pip-install-z65rvnuy/ghclone
  Running command git clone -q git://github.com/HR/github-clone /tmp/pip-install-z65rvnuy/ghclone
  Created wheel for ghclone: filename=ghclone-1.2.0-cp36-none-any.whl size=8903 sha256=1a8f81da4f56f0076ca9505ac20a7c1f17e2dbc64ac7a3c8bd3c5cdc350a7d25
  Stored in directory: /tmp/pip-ephem-wheel-cache-8cr1ptly/wheels/96/ef/52/b380c74df9290d33a3f52e0d38792869a7f07bb5e208805916
Successfully built ghclone


In [2]:
!ghclone https://github.com/google-research/google-research/tree/master/tft

Cloning into 'tft'...
done.


In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
cd tft

/content/tft


# train

In [0]:
import argparse
import datetime as dte
import os

import data_formatters.base
import expt_settings.configs
import libs.hyperparam_opt
import libs.tft_model
import libs.utils as utils
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf

In [0]:
ExperimentConfig = expt_settings.configs.ExperimentConfig
HyperparamOptManager = libs.hyperparam_opt.HyperparamOptManager
ModelClass = libs.tft_model.TemporalFusionTransformer

In [0]:
# where $EXPT can be any of {volatility, electricity, traffic, favorita}, 
# and $OUTPUT_FOLDER denotes the root folder in which experiment outputs are saved.
EXPT='favorita'
OUTPUT_FOLDER='favorita_test'

In [7]:
# args for electricity testing from gpu
name, output_folder, use_tensorflow_with_gpu = EXPT, OUTPUT_FOLDER, True

print("Using output folder {}".format(output_folder))

config = ExperimentConfig(name, output_folder)
formatter = config.make_data_formatter()

Using output folder favorita_test


In [0]:
expt_name=name
use_gpu=use_tensorflow_with_gpu
model_folder=os.path.join(config.model_folder, "fixed")
data_csv_path=config.data_csv_path
data_formatter=formatter
#use_testing_mode=False

In [0]:
#num_repeats = 1

if not isinstance(data_formatter, data_formatters.base.GenericDataFormatter):
  raise ValueError(
      "Data formatters should inherit from" +
      "AbstractDataFormatter! Type={}".format(type(data_formatter)))

In [0]:
# Tensorflow setup
default_keras_session = tf.keras.backend.get_session()

In [11]:
if use_gpu:
  tf_config = utils.get_default_tensorflow_config(tf_device="gpu", gpu_id=0)

else:
  tf_config = utils.get_default_tensorflow_config(tf_device="cpu")

print("*** Training from defined parameters for {} ***".format(expt_name))

Selecting GPU ID=0

*** Training from defined parameters for favorita ***


In [13]:
!pip install ipython-autotime

%load_ext autotime

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=c6991e93c839e455c87760925d7feb7617012993a6c5fa593d7abc37079478aa
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


# GCP Bucket에서 가져오기

In [14]:
from google.colab import auth
auth.authenticate_user()

time: 19.7 s


##### each dataset import ver.

In [0]:
#!gsutil cp gs://iskra3138_retail/temp/tft/favorita_train_fn.csv /content

Copying gs://iskra3138_retail/temp/tft/favorita_train_fn.csv...
\ [1 files][  3.5 GiB/  3.5 GiB]   69.6 MiB/s                                   
Operation completed over 1 objects/3.5 GiB.                                      
time: 1min 10s


In [0]:
#!gsutil cp gs://iskra3138_retail/temp/tft/favorita_valid_fn.csv /content

Copying gs://iskra3138_retail/temp/tft/favorita_valid_fn.csv...
\ [1 files][  1.6 GiB/  1.6 GiB]   69.6 MiB/s                                   
Operation completed over 1 objects/1.6 GiB.                                      
time: 40.3 s


In [0]:
#!gsutil cp gs://iskra3138_retail/temp/tft/favorita_test_fn.csv /content

Copying gs://iskra3138_retail/temp/tft/favorita_test_fn.csv...
/ [1 files][225.2 MiB/225.2 MiB]                                                
Operation completed over 1 objects/225.2 MiB.                                    
time: 6.88 s


In [0]:
#raw_train = pd.read_csv('/content/favorita_train_fn.csv', index_col=0)
#raw_train['date'] = pd.to_datetime(raw_train['date'])

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


time: 51.5 s


In [0]:
#raw_valid = pd.read_csv('/content/favorita_valid_fn.csv', index_col=0)
#raw_valid['date'] = pd.to_datetime(raw_valid['date'])

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


time: 21.2 s


In [0]:
#raw_test = pd.read_csv('/content/favorita_test_fn.csv', index_col=0)
#raw_test['date'] = pd.to_datetime(raw_test['date'])

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


time: 3.04 s


In [0]:
#raw_data = pd.concat([raw_train, raw_valid, raw_test])

time: 6.02 s


In [0]:
#print (len(raw_train), len(raw_valid), len(raw_test))

22136420 9800160 1388640
time: 1.09 ms


In [0]:
#data_formatter.set_scalers(raw_train, set_real=True)

Setting scalers with training data...
time: 1.07 s


In [0]:
#data_formatter.set_scalers(raw_data, set_real=False)

Setting scalers with training data...
time: 2min 9s


In [0]:
import gc

time: 974 µs


In [0]:
del raw_data
gc.collect()

0

time: 563 ms


In [0]:
#train = data_formatter.transform_inputs(raw_train)

time: 1min 41s


In [0]:
#del raw_train
#gc.collect()

0

time: 311 ms


In [0]:
#valid = data_formatter.transform_inputs(raw_valid)

time: 44.5 s


In [0]:
#del raw_valid
#gc.collect()

0

time: 191 ms


In [0]:
#test = data_formatter.transform_inputs(raw_test)

time: 6 s


In [0]:
#del raw_test
#gc.collect()

0

time: 102 ms


##### full dataset import ver.

In [15]:
!gsutil cp gs://iskra3138_retail/temp/tft/favorita_test/data/favorita/favorita_consolidated.csv /content

Copying gs://iskra3138_retail/temp/tft/favorita_test/data/favorita/favorita_consolidated.csv...
| [1 files][  8.3 GiB/  8.3 GiB]   65.2 MiB/s                                   
Operation completed over 1 objects/8.3 GiB.                                      
time: 3min 18s


In [0]:
raw_data = pd.read_csv('/content/favorita_consolidated.csv', index_col=0)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,6,23,24) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [0]:
# train, valid, test = data_formatter.split_data(raw_data)

In [0]:
valid_boundary = pd.datetime(2015, 12, 1)
fixed_params = data_formatter.get_fixed_params()
time_steps = fixed_params['total_time_steps']
lookback = fixed_params['num_encoder_steps']
forecast_horizon = time_steps - lookback
raw_data['date'] = pd.to_datetime(raw_data['date'])
df_lists = {'train': [], 'valid': [], 'test': []}

In [0]:
for _, sliced in raw_data.groupby('traj_id'):
  index = sliced['date']
  train = sliced.loc[index < valid_boundary]
  train_len = len(train)
  valid_len = train_len + forecast_horizon
  valid = sliced.iloc[train_len - lookback:valid_len, :]
  test = sliced.iloc[valid_len - lookback:valid_len + forecast_horizon, :]

  sliced_map = {'train': train, 'valid': valid, 'test': test}

  for k in sliced_map:
    item = sliced_map[k]

    if len(item) >= time_steps:
      df_lists[k].append(item)

In [0]:
dfs = {k: pd.concat(df_lists[k], axis=0) for k in df_lists}

train = dfs['train']
data_formatter.set_scalers(train, set_real=True)

# Use all data for label encoding  to handle labels not present in training.
data_formatter.set_scalers(raw_data, set_real=False)

In [0]:
import gc
del raw_data
gc.collect()

In [0]:
valid = filter_ids(dfs['valid'])
test = filter_ids(dfs['test'])
train, valid, test = data_formatter.transform_inputs(data) for data in [train, valid, test]

In [0]:
train_samples, valid_samples = data_formatter.get_num_samples_for_calibration()

time: 844 µs


In [0]:
# Sets up default params
fixed_params = data_formatter.get_experiment_params()
params = data_formatter.get_default_model_params()
params["model_folder"] = model_folder

time: 1.34 ms


In [0]:
# Sets up hyperparam manager
print("*** Loading hyperparm manager ***")
opt_manager = HyperparamOptManager({k: [params[k]] for k in params},
                                    fixed_params, model_folder)

*** Loading hyperparm manager ***
time: 3.88 ms


In [0]:
# Training -- one iteration only
print("*** Running calibration ***")
print("Params Selected:")
for k in params:
  print("{}: {}".format(k, params[k]))

*** Running calibration ***
Params Selected:
dropout_rate: 0.1
hidden_layer_size: 240
learning_rate: 0.001
minibatch_size: 128
max_gradient_norm: 100.0
num_heads: 4
stack_size: 1
model_folder: favorita_test/saved_models/favorita/fixed
time: 1.82 ms


In [0]:
params = opt_manager.get_next_parameters()

time: 13.9 ms


Next, we train a TFT model by subsampling the data in the training and validation sets.

In [0]:
tf.reset_default_graph()
with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    
    # Create a TFT model
    model = ModelClass(params, use_cudnn=True) # Run model on GPU using CuDNNLSTM cells

    # Sample data into minibatches for training
    if not model.training_data_cached():
        model.cache_batched_data(train, "train", num_samples=100000)
        model.cache_batched_data(valid, "valid", num_samples=50000)

    # Train and save model
    model.fit()
    model.save(model_folder)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Getting locations for 7_1642398
Getting locations for 7_1642399
Getting locations for 7_1642400
Getting locations for 7_1642401
Getting locations for 7_1642664
Getting locations for 7_164647
Getting locations for 7_165550
Getting locations for 7_165551
Getting locations for 7_165553
Getting locations for 7_165594
Getting locations for 7_165693
Getting locations for 7_165704
Getting locations for 7_165705
Getting locations for 7_165727
Getting locations for 7_1658994
Getting locations for 7_165988
Getting locations for 7_1660188
Getting locations for 7_1660191
Getting locations for 7_1660193
Getting locations for 7_1660194
Getting locations for 7_1660239
Getting locations for 7_1660260
Getting locations for 7_1660261
Getting locations for 7_1660271
Getting locations for 7_1660272
Getting locations for 7_1672581
Getting locations for 7_1673295
Getting locations for 7_1673297
Getting locations for 7_167437
Getting locations for 7_1686644
Getting locatio

To evaluate model performance, we reload the serialised model and compute P50 and P90 losses.

In [0]:
tf.reset_default_graph()
with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    
    # Create a new model & load weights
    model = ModelClass(params, use_cudnn=True)
    model.load(model_folder)
    
    # Make forecasts
    output_map = model.predict(test, return_targets=True)

    targets = data_formatter.format_predictions(output_map["targets"])

    # Format predictions
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
          col for col in data.columns
          if col not in {"forecast_time", "identifier"}
        ]]

    # Compute Losses
    p50_loss = utils.numpy_normalised_quantile_loss(
        extract_numerical_data(targets), extract_numerical_data(p50_forecast),
        0.5)
    p90_loss = utils.numpy_normalised_quantile_loss(
        extract_numerical_data(targets), extract_numerical_data(p90_forecast),
        0.9)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 -3.09489705e-02 -2.12919284e-02 -1.86617989e-02 -3.78850400e-02
 -5.56315370e-02 -6.23204093e-03  9.32329241e-03 -2.62383483e-02
 -8.59739073e-03 -2.51793526e-02 -6.02650084e-02  3.89342830e-02
 -7.68647268e-02 -1.04844548e-04  3.50500979e-02 -3.73505242e-02
 -5.61764203e-02  1.00336941e-02  2.00195424e-02 -1.41948396e-02
  1.19935744e-01 -3.93578932e-02 -1.76320411e-02 -2.81399433e-02
  6.82955980e-02  6.27704803e-03  6.35658903e-03 -4.01047915e-02
 -1.93943046e-02 -4.32035252e-02 -1.78690087e-02  1.85180586e-02
  8.29814840e-03  1.30528351e-02 -5.29883131e-02 -5.48261106e-02
 -4.37244549e-02 -4.13172506e-02 -6.37397319e-02 -6.27339408e-02
 -4.98892330e-02  1.40749170e-02 -2.82110199e-02  6.88490923e-03
 -3.07218693e-02  9.65956599e-03  3.60339805e-02 -5.44961169e-02
 -5.85498028e-02 -4.07859646e-02 -3.81457210e-02 -1.34795094e-02
 -4.18815166e-02 -3.19490582e-02 -4.14163731e-02 -6.46827072e-02
  5.06522059e-02  2.63593607e-02  1.00581152e-02 -3.64

In [0]:
print("Normalised quantile losses: P50={}, P90={}".format(p50_loss.mean(), p90_loss.mean()))

Normalised quantile losses: P50=0.3360804327506121, P90=0.14445513654933914
time: 1.5 ms


## Interpretability Use Cases
The relationships learnt by the TFT can also be studied using the trained model, through:

1. Analyzing the variable selection weights to identify significant features for the prediction problem.
2. Visualizing distributions of self-attention weights to determine the presence of any persistent temporal relationships.

In the remainder of this section, we demonstrate two interpretability use cases to showcase the above.

### Generate Weights for Interpretability
First, we generate all necessary variable selection and attention weights required for analysis.

In [0]:
# Store outputs in maps
counts = 0
interpretability_weights = {k: None for k in ['decoder_self_attn', 
                                              'static_flags', 'historical_flags', 'future_flags']}

tf.reset_default_graph()
with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    
    # Create a new model & load weights
    model = ModelClass(params, 
                                    use_cudnn=True)
    model.load(model_folder)
    for identifier, sliced in test.groupby('traj_id'):
        
        print("Getting attention weights for {}".format(identifier))
        weights = model.get_attention(sliced)
        
        for k in interpretability_weights:
            w = weights[k]
            
            # Average attentin across heads if necessary
            if k == 'decoder_self_attn':
                w = w.mean(axis=0)
            
                # Store a single matrix for weights to reduce memory footprint
                batch_size, _, _ = w.shape                 
                counts += batch_size
            
            if interpretability_weights[k] is None:
                interpretability_weights[k] = w.sum(axis=0)
            else:
                interpretability_weights[k] += w.sum(axis=0)

interpretability_weight = {k: interpretability_weights[k]/counts for k in interpretability_weights}

print('Done.')

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
   0.00594103]
 [ 0.00378726  0.04894188 -0.03154607 ... -0.09501971  0.12360397
  -0.00322568]
 ...
 [-0.12562133 -0.04412106 -0.02893431 ... -0.07334027  0.24671729
   0.07790156]
 [ 0.17260598  0.06131192 -0.09956042 ... -0.01664287 -0.01053553
   0.05038585]
 [-0.03069553  0.0087095  -0.0712318  ... -0.04053007 -0.13803358
  -0.00579988]]
tensor_name:  TemporalFusionTransformer/time_distributed_67/bias
[ 0.01946679 -0.01898478 -0.08332723 -0.00681202 -0.04756457 -0.15155014
  0.11448463 -0.12071174  0.00182302 -0.04077926 -0.13143224  0.00491782
 -0.00159931  0.00433443 -0.00459724  0.01303445  0.063203    0.00927456
 -0.0708049  -0.01666487 -0.07344899  0.02179372 -0.05540891 -0.01600887
  0.01985352  0.13938542  0.07808566 -0.01089944 -0.04475059 -0.05810319
  0.08994815  0.07801502  0.02966555 -0.0437106   0.01127718 -0.03881334
 -0.00908395 -0.01196778  0.03641704  0.02141306 -0.05723217 -0.1377411
 -0.0321286   0.01290016  0.13775171 -0.1145

### Use Case 1: Analyzing Variable Importance 

Next, we analyze the distribution of variable selection weights on the input layer -- using this to quantify the relative importance of a given feature for the prediction problem in general. This is split into variable importance for static covariates, time-varying historical inputs variables and known future inputs as shown below.

In [0]:
import numpy as np
def get_range(static_gate, axis=None):
    """Returns the mean, 10th, 50th and 90th percentile of variable importance weights."""
    return {'Mean': static_gate.mean(axis=axis), 
               '10%': np.quantile(static_gate, 0.1, axis=axis),
               '50%': np.quantile(static_gate, 0.5, axis=axis),
               '90%': np.quantile(static_gate, 0.9, axis=axis)}

#### Static Variable Importance

In [0]:
def flatten(x):
    static_attn = x
    static_attn = static_attn.reshape([-1, static_attn.shape[-1]])
    return static_attn

static_attn = flatten(interpretability_weights['static_flags'])
m = get_range(static_attn, axis=0)
pd.DataFrame({k: pd.Series(m[k], index=['ID']) for k in m})

#### Temporal Variable Importance -- Past Inputs

In [0]:
x = flatten(interpretability_weights['historical_flags'])
m = get_range(x, axis=0)
pd.DataFrame({k: pd.Series(m[k], index=['Hour of Day', 'Day of Week', 'Time Index', 'Target']) for k in m})

#### Temporal Variable Importance -- Future Inputs

In [0]:
x = flatten(interpretability_weights['future_flags'])
m = get_range(x, axis=0)
pd.DataFrame({k: pd.Series(m[k], index=['Hour of Day', 'Day of Week', 'Time Index']) for k in m})

### Use Case 2: Visualizing Persistent Temporal Patterns
Lastly, we analyse the distribution of self-attention weights across various horizons to see if any persistent temporal patterns exist within the dataset. This allows us to identify any seasonal patterns or lagged relationships in the dataset, based on which past time steps are consistently important for predictions at a given horizon. 

We visualize this using the average attention pattern for various prediction horizons, as shown below:

#### Mean Attention Weights for Various Prediction Horizons

In [0]:
# Plotting libraries & Functions
import plotly.offline
from plotly.offline import download_plotlyjs, init_notebook_mode, plot
import plotly.graph_objs as go
import cufflinks as cf
from IPython.display import HTML

# Loads plotly charts
def iplot(fig, s='plot.html'):
    filename = os.path.join(output_folder, s)
    plotly.offline.plot(fig, filename=filename, auto_open=False)
    return HTML(filename)    

def plotly_chart(df, title=None, kind='scatter', x_label=None, y_label=None, secondary_y=None, fill=None,
                shape=None, subplots=False):
    
    fig = df.iplot(asFigure=True, title=title, kind=kind, xTitle=x_label, yTitle=y_label, secondary_y=secondary_y,
                  fill=fill, subplots=subplots, shape=shape)

    return iplot(fig)

In [0]:
self_attn = interpretability_weights['decoder_self_attn']

means = pd.DataFrame({"horizon={}".format(k): self_attn[model.num_encoder_steps+k-1, :] 
                      for k in [1, 5, 10, 15, 20]})
means.index -= model.num_encoder_steps

plotly_chart(means,
             x_label="Positiion Index (n)",
             y_label="Mean Attention Weight",
             title="Average Attention Pattern at Various Prediction Horizons")